In [155]:
import os 
import google.auth

from google.oauth2 import service_account

from config import config_env

from function.function import  FromDriveToStorage
import google.auth


CONFIG_ENV = os.getenv("ENV") or "dev"
config = config_env[CONFIG_ENV]

# Create credentials with Drive & BigQuery API scopes.
# Both APIs must be enabled for your project before running this code.
# credentials, project = google.auth.default(
#     scopes=config.SCOPES
# )

key_path='/home/portfolio/GCP-Templates/cloud_function/from_drive_to_storage/function/credentials.json'
creds=None
if CONFIG_ENV=="dev":
     creds = service_account.Credentials.from_service_account_file(key_path) # If you're using local machine    
     scoped_credentials = creds.with_scopes(config.SCOPES)    
# else:
# creds, _ = google.auth.default()
# scoped_credentials = creds.with_scopes(config.SCOPES)
# 

FromDriveToStorage(config, creds=creds, scoped_credentials=scoped_credentials)


MalformedError: Service account info was not in the expected format, missing fields client_email, token_uri.

In [159]:
import google.oauth2.credentials

key_path='/home/portfolio/GCP-Templates/cloud_function/from_drive_to_storage/function/drive.json'
credentials = google.oauth2.credentials.Credentials(key_path)

In [157]:
credentials = google.oauth2.credentials.Credentials(
    "d4PMufmHKVwcPc1IY40n0UnduTfjTd",
    refresh_token=None,
    token_uri="https://oauth2.googleapis.com/token",
    client_id='318040672941-jbi4pi3rhckhlcpv5cp32v0c01lofqml.apps.googleusercontent.com',
    client_secret='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')
credentials

In [161]:
import google.oauth2.credentials
import google_auth_oauthlib.flow

# Use the client_secret.json file to identify the application requesting
# authorization. The client ID (from that file) and access scopes are required.
key_path='/home/portfolio/GCP-Templates/cloud_function/from_drive_to_storage/function/credentials.json'

flow = google_auth_oauthlib.flow.Flow.from_client_secrets_file(
    key_path,
    scopes=['https://www.googleapis.com/auth/drive.metadata.readonly'])

# Indicate where the API server will redirect the user after the user completes
# the authorization flow. The redirect URI is required. The value must exactly
# match one of the authorized redirect URIs for the OAuth 2.0 client, which you
# configured in the API Console. If this value doesn't match an authorized URI,
# you will get a 'redirect_uri_mismatch' error.
flow.redirect_uri = 'https://www.example.com/oauth2callback'

# Generate URL for request to Google's OAuth 2.0 server.
# Use kwargs to set optional request parameters.
authorization_url, state = flow.authorization_url(
    # Enable offline access so that you can refresh an access token without
    # re-prompting the user for permission. Recommended for web server apps.
    access_type='offline',
    # Enable incremental authorization. Recommended as a best practice.
    include_granted_scopes='true')

In [163]:
from googleapiclient.discovery import build

from google_auth_oauthlib.flow import InstalledAppFlow

flow = InstalledAppFlow.from_client_secrets_file(
                key_path, config().SCOPES)
creds = flow.run_local_server(port=0)
service=build('drive', 'v3', credentials=creds)
results = service.files().list(
        pageSize=5, fields="nextPageToken, files(id, name, mimeType, size, parents, modifiedTime)").execute()
    # get the results
items = results.get('files', [])
    # list all 20 files & folders
items

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=318040672941-jbi4pi3rhckhlcpv5cp32v0c01lofqml.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A52287%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.file+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.readonly+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.metadata.readonly+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.appdata+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.metadata+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly&state=hxbcRdEeO9sy4GiG5uBbFWx3Xo99hL&access_type=offline


KeyboardInterrupt: 

In [151]:


import pickle
import os
from googleapiclient.discovery import build

# If modifying these scopes, delete the file token.pickle.
SCOPES = ['https://www.googleapis.com/auth/drive.metadata.readonly']


def get_gdrive_service():
    creds = None
    # The file token.pickle stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            # creds = flow.run_local_server(port=0)
            name ="a"            
            storage = file.Storage(name + ".dat")
            creds = tools.run_flow(flow, storage)

        # Save the credentials for the next run
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)
    # return Google Drive API service
    return build('drive', 'v3', credentials=creds)




"""Shows basic usage of the Drive v3 API.
Prints the names and ids of the first 5 files the user has access to.
"""
service = get_gdrive_service()
# Call the Drive v3 API
results = service.files().list(
    pageSize=1, fields="nextPageToken, files(id, name, mimeType, size, parents, modifiedTime)").execute()
# get the results
items = results.get('files', [])
# list all 20 files & folders
items


usage: ipykernel_launcher.py [--auth_host_name AUTH_HOST_NAME]
                             [--noauth_local_webserver]
                             [--auth_host_port [AUTH_HOST_PORT ...]]
                             [--logging_level {DEBUG,INFO,WARNING,ERROR,CRITICAL}]
ipykernel_launcher.py: error: unrecognized arguments: --ip=127.0.0.1 --stdin=9003 --control=9001 --hb=9000 --Session.signature_scheme="hmac-sha256" --Session.key=b"b270b80e-5bfe-4896-b25d-75600a07d243" --shell=9002 --transport="tcp" --iopub=9004 --f=/root/.local/share/jupyter/runtime/kernel-v2-307b6m6plQsgjoG.json


SystemExit: 2

/home/gcp-templates/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3468: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [166]:
import os
from google.oauth2 import service_account
key_path='/home/portfolio/GCP-Templates/cloud_function/from_drive_to_storage/function/drive.json'
creds = service_account.Credentials.from_service_account_file(key_path) # If you're using local machine    

SCOPES = ['https://www.googleapis.com/auth/drive.metadata.readonly']
scoped_credentials = creds.with_scopes(SCOPES)    
request = requests.Request()

# Refresh the default credentials. This ensures that the information
# about this account, notably the email, is populated.
# credentials.refresh(request)
# creds, _ = google.auth.default()
# scoped_credentials = creds.with_scopes(config.SCOPES)

# creds = service_account.Credentials.from_service_account_file(key_path) # If you're using local machine    

# service_account_info = json.loads(os.getenv('GDRIVE_AUTH'))
SCOPES = ['https://www.googleapis.com/auth/drive.metadata.readonly']

# creds = service_account.Credentials.from_service_account_info(
#         service_account_info, scopes=SCOPES)

service = build('drive', 'v3', credentials=scoped_credentials)
results = service.files().list(
        pageSize=5, fields="nextPageToken, files(id, name, mimeType, size, parents, modifiedTime)").execute()
    # get the results
items = results.get('files', [])
    # list all 20 files & folders
items

[{'mimeType': 'application/vnd.google-apps.folder',
  'id': '1uymD6k_4JTTP4YuTP_NBTAftS4nteU5o',
  'name': 'germination_analysis',
  'modifiedTime': '2023-03-03T23:34:10.600Z'},
 {'mimeType': 'text/csv',
  'parents': ['1uymD6k_4JTTP4YuTP_NBTAftS4nteU5o'],
  'size': '32036',
  'id': '1kCoIzOE9Sj3xYporr1CEjXn7_AKwrORp',
  'name': 'ABA.csv',
  'modifiedTime': '2023-02-22T21:19:10.000Z'}]

In [ ]:
from oauth2client.client import OAuth2WebServerFlow
from oauth2client.tools import run_flow
from oauth2client.file import Storage

CLIENT_ID = '318040672941-jbi4pi3rhckhlcpv5cp32v0c01lofqml.apps.googleusercontent.com'
CLIENT_SECRET = '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'
flow = OAuth2WebServerFlow(client_id=CLIENT_ID,
                    client_secret=CLIENT_SECRET,
                    scope='https://spreadsheets.google.com/feeds https://docs.google.com/feeds',
                    redirect_uri='http://example.com/auth_return',
                    prompt='consent',
                    access_type='offline')

# flow = InstalledAppFlow.from_client_secrets_file(
#         'credentials.json', scope=SCOPES,       
#         redirect_uri='http://example.com/auth_return',
#         prompt='consent')

storage = Storage('creds.data')
credentials = run_flow(flow, storage)
credentials

In [ ]:
os.path.dirname("credentials")

In [ ]:
credential_dir = os.path.join(os.getcwd(), '.credentials')
if not os.path.exists(credential_dir):
    os.makedirs(credential_dir)
credential_path = os.path.join(credential_dir,
                                'gmail-quickstart.json')

store = oauth2client.file.Storage(credential_path)
credentials = store.get()
if not credentials or credentials.invalid:
    flow = client.flow_from_clientsecrets(CLIENT_SECRET_FILE, SCOPES)
    flow.user_agent = APPLICATION_NAME
    flags = argparse.Namespace(auth_host_name='localhost',
                                auth_host_port=[8080, 8090],
                                logging_level='ERROR',
                                noauth_local_webserver=False)
    credentials = tools.run_flow(flow, store, flags)
    print 'Storing credentials to ' + credential_path

In [ ]:
store = Storage("credentials.json")
credentials = store.get()
if not credentials or credentials.invalid:
    credentials = run_flow(flow, store)
service = build("gmail", "v1", http=credentials.authorize(build_http()))


In [149]:
creds, _ = google.auth.default()


DefaultCredentialsError: Could not automatically determine credentials. Please set GOOGLE_APPLICATION_CREDENTIALS or explicitly create credentials and re-run the application. For more information, please see https://cloud.google.com/docs/authentication/getting-started

In [150]:
import argparse
import os

from googleapiclient import discovery
from googleapiclient.http import build_http
from oauth2client import client, file, tools

client_secrets = os.path.join(os.path.dirname("credentials"), "credentials.json")
SCOPES = ['https://www.googleapis.com/auth/drive.metadata.readonly']
name='drive'
version="3"

# flags = tools.argparser.parse_args(args=['--noauth_local_webserver'])
flags = argparse.Namespace(auth_host_name='localhost',
                                auth_host_port=[8080, 8090],
                                logging_level='ERROR',
                                noauth_local_webserver=False)


# Set up a Flow object to be used if we need to authenticate.
flow = client.flow_from_clientsecrets(
    client_secrets, scope=SCOPES
)
storage = file.Storage(name + ".dat")
credentials = storage.get()
if credentials is None or credentials.invalid:
    credentials = tools.run_flow(flow, storage, flags)
http = credentials.authorize(http=build_http())

service = discovery.build(name, version, http=http)




Failed to start a local webserver listening on either port 8080
or port 8090. Please check your firewall settings and locally
running programs that may be blocking or using those ports.

Falling back to --noauth_local_webserver and continuing with
authorization.


Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?client_id=318040672941-jbi4pi3rhckhlcpv5cp32v0c01lofqml.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.metadata.readonly&access_type=offline&response_type=code



AttributeError: 'tuple' object has no attribute 'tb_frame'

In [ ]:
results = service.files().list(
        pageSize=5, fields="nextPageToken, files(id, name, mimeType, size, parents, modifiedTime)").execute()
    # get the results
items = results.get('files', [])
    # list all 20 files & folders
items

In [ ]:
from apiclient.discovery import build
from oauth2client.service_account import ServiceAccountCredentials
from httplib2 import Http

scopes = ['https://www.googleapis.com/auth/drive.readonly&#39']
key_path='/home/portfolio/GCP-Templates/cloud_function/from_drive_to_storage/function/key_storage.json'

import google.oauth2.credentials
credentials = google.oauth2.credentials.Credentials(key_path)
http_auth = credentials.authorize(Http())
drive = build('drive', 'v3', http=http_auth)

request = drive.files().list().execute()
files = request.get('items', [])
for f in files:
    print(f)

In [ ]:
from __future__ import print_function

import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# If modifying these scopes, delete the file token.json.
SCOPES = ["https://www.googleapis.com/auth/drive.readonly"]



creds = None
# The file token.json stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists('token.json'):
    creds = Credentials.from_authorized_user_file('token.json', config().SCOPES)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        # flow = InstalledAppFlow.from_client_secrets_file(
        #     'credentials.json', SCOPES)
        flow = InstalledAppFlow.from_client_secrets_file(
                key_path, config().SCOPES)

        creds = flow.run_local_server(port=0)
        name ="a"            
        storage = file.Storage(name + ".dat")
        creds = tools.run_flow(flow, storage)

    # Save the credentials for the next run
    with open('token.json', 'w') as token:
        token.write(creds.to_json())

try:
    service = build('drive', 'v3', credentials=creds)

    # Call the People API
    print('List 10 connection names')
    results = service.files().list(
        pageSize=1, fields="nextPageToken, files(id, name, mimeType, size, parents, modifiedTime)").execute()
    # get the results
    items = results.get('files', [])
    # list all 20 files & folders
    connections = results.get('connections', [])

    for person in connections:
        names = person.get('names', [])
        if names:
            name = names[0].get('displayName')
            print(name)
except HttpError as err:
    print(err)




In [ ]:
# Llama al API para listar los archivos
results = service.files().list(
    pageSize=10, fields="nextPageToken, files(id, name)").execute()
items = results.get('files', [])

# Imprime el resultado
if not items:
    print('No se encontraron archivos.')
else:
    print('Archivos:')
    for item in items:
        print(f'{item["name"]} ({item["id"]})')
